In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q
from apps.environment_simulator.sevice_layer.market_simulator import MarketSimulator
from apps.environment_simulator.models import SimulatedStockBuffer
import gymnasium as gym
import ray
from gymnasium import spaces
from ray.rllib.algorithms import ppo
from os import path, environ

2023-10-07 20:14:28,818	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}

config = {
        "env": MarketSimulator,
        "env_config": {"db_params": db_params},
        "num_workers": 4,
}

ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})
algo = ppo.PPO(config=config)

2023-10-07 20:14:32,568	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-07 20:14:32,936	INFO packaging.py:518 -- Creating a file package for local directory '/Users/mirbilal/Desktop/MobCommission/commissionV2/'.
2023-10-07 20:14:33,190	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_31ebffced6f562bb.zip' (6.55MiB) to Ray cluster...
2023-10-07 20:14:33,221	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_31ebffced6f562bb.zip'.
2023-10-07 20:14:35,647	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-10-07 20:14:35,648	WARNING algorithm_config.py:672 -- Cannot create PPOConfi

RaySystemError: System error: Apps aren't loaded yet.
traceback: Traceback (most recent call last):
  File "/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/ray/_private/serialization.py", line 404, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
  File "/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/ray/_private/serialization.py", line 270, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/ray/_private/serialization.py", line 225, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/ray/_private/serialization.py", line 215, in _deserialize_pickle5_data
    obj = pickle.loads(in_band)
  File "/tmp/ray/session_2

In [4]:
ray.shutdown()

In [4]:
mean_ppo = []
for _ in range(50):

    result = algo.train()
    print("episode reward mean:", _, result['episode_reward_mean'])
    mean_ppo.append(result['episode_reward_mean'])


episode reward mean: 0 -0.991
episode reward mean: 1 -0.991
episode reward mean: 2 -0.991
episode reward mean: 3 -0.9845
episode reward mean: 4 -0.975
episode reward mean: 5 -0.969
episode reward mean: 6 -0.9535
episode reward mean: 7 -0.941
episode reward mean: 8 -0.925
episode reward mean: 9 -0.9155
episode reward mean: 10 -0.8895
episode reward mean: 11 -0.871
episode reward mean: 12 -0.8525
episode reward mean: 13 -0.839
episode reward mean: 14 -0.83
episode reward mean: 15 -0.7985
episode reward mean: 16 -0.7765
episode reward mean: 17 -0.763
episode reward mean: 18 -0.732
episode reward mean: 19 -0.6855
episode reward mean: 20 -0.6535
episode reward mean: 21 -0.612
episode reward mean: 22 -0.5745
episode reward mean: 23 -0.5165
episode reward mean: 24 -0.466
episode reward mean: 25 -0.4075
episode reward mean: 26 -0.314
episode reward mean: 27 -0.205
episode reward mean: 28 -0.0895
episode reward mean: 29 0.002
episode reward mean: 30 0.0945
episode reward mean: 31 0.2065
episode

In [15]:
MyEnv()
obs = MyEnv()
obs.reset()

latest_action = algo.compute_single_action(0)
print(latest_action)

[1. 1. 2. 0. 1.]


/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning:

WARN: Box bound precision lowered by casting to float32



In [5]:
from gymnasium import spaces

no_of_stocks = 5
stock_observation_shape = (no_of_stocks, 8)

stock_lower_bounds = np.zeros(stock_observation_shape)
stock_upper_bounds = np.full(stock_observation_shape, float('inf'))

no_of_cmmdts = 6
commodity_observation_shape = (no_of_cmmdts, 1) 

commodity_lower_bounds = np.zeros(commodity_observation_shape)
commodity_upper_bounds = np.full(commodity_observation_shape, float('inf'))

stock_observation_space = spaces.Box(low=stock_lower_bounds, high=stock_upper_bounds)
commodity_observation_space = spaces.Box(low=commodity_lower_bounds, high=commodity_upper_bounds)

wallet_observation_space = spaces.Box(low=0, high=float('inf'), shape=(1,))
states = spaces.Tuple((stock_observation_space, commodity_observation_space, wallet_observation_space))

print(states)

Tuple(Box(0.0, inf, (5, 8), float32), Box(0.0, inf, (6, 1), float32), Box(0.0, inf, (1,), float32))
